# **Required Packages**

In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import  DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import sqlite3
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# **Data collection**

In [ ]:
# Reading data from drive

city_df = pd.read_excel('/content/drive/MyDrive/mini_project_4_db/City.xlsx')

continent_df = pd.read_excel("/content/drive/MyDrive/mini_project_4_db/Continent.xlsx")

country_df = pd.read_excel("/content/drive/MyDrive/mini_project_4_db/Country.xlsx")

item_df = pd.read_excel("/content/drive/MyDrive/mini_project_4_db/Updated_Item.xlsx")

mode_df = pd.read_excel("/content/drive/MyDrive/mini_project_4_db/Mode.xlsx")

region_df = pd.read_excel("/content/drive/MyDrive/mini_project_4_db/Region.xlsx")

transaction_df = pd.read_excel("/content/drive/MyDrive/mini_project_4_db/Transaction.xlsx")

type_df = pd.read_excel("/content/drive/MyDrive/mini_project_4_db/Type.xlsx")

user_df = pd.read_excel("/content/drive/MyDrive/mini_project_4_db/User.xlsx")


In [ ]:
# Merging Dataframes

df_merged1 = pd.merge(transaction_df, user_df, on='UserId', how='left')
df_merged2 = pd.merge(df_merged1, region_df[['Region','RegionId']], on='RegionId', how='left')
df_merged3 = pd.merge(df_merged2, item_df, on='AttractionId', how='left')
df_merged4 = pd.merge(df_merged3, country_df[['CountryId','Country']], on='CountryId', how='left')
df_merged5 = pd.merge(df_merged4, continent_df, on='ContinentId', how='left')
df_merged6 = pd.merge(df_merged5, city_df[['CityId','CityName']], on='CityId', how='left')
df_merged7 = pd.merge(df_merged6, type_df, on='AttractionTypeId', how='left')
df_merged7.rename(columns={'VisitMode': 'VisitModeId'}, inplace=True)
main_data = pd.merge(df_merged7, mode_df, on='VisitModeId', how='left')

In [ ]:
#Checking Na values

main_data.isna().sum()

In [ ]:
main_data.describe()

In [ ]:
main_data.dropna(inplace=True)

In [ ]:
main_data.info()

# **Transferring Data to SQL**

In [ ]:
import sqlite3
conn = sqlite3.connect('my_database.db')  # .db file will be created
cursor = conn.cursor()
main_data.to_sql('tourist_data', conn, if_exists='append', index=False)

# **Pre-proccessig**

In [ ]:
# Dropping unwanted columns

main_data_ml = main_data.drop('AttractionAddress',axis=1)

In [ ]:
#Encoding columns with object datatype

categorical_cols = main_data_ml.select_dtypes(include='object').columns.tolist()
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    main_data_ml[col] = le.fit_transform(main_data_ml[col])
    label_encoders[col] = le

In [ ]:
# save the encoder

with open("label_encoder.pkl",'wb') as encoder_file:
  pickle.dump(label_encoders,encoder_file)

# **EDA**

In [ ]:
#Distribution across continents, countries, and regions.

In [ ]:
distribution_continent= main_data['Continent'].value_counts().reset_index(name='Visit_count')
fig = px.bar(distribution_continent,'Continent','Visit_count',title="Distribution across Continent",color='Continent')
fig.show()

In [ ]:
distribution_country = main_data['Country'].value_counts().reset_index(name='Visit_count')
fig = px.bar(distribution_country,'Country','Visit_count',title="Distribution across country",color='Country')
fig.show()


In [ ]:
distribution_region= main_data['Region'].value_counts().reset_index(name='Visit_count')
fig = px.bar(distribution_region,'Region','Visit_count',title="Distribution across Region",color='Region')
fig.show()

In [ ]:
#attraction types and their popularity based on user ratings

popular_att = main_data[['AttractionType','Rating']]

In [ ]:
pop_att = popular_att.groupby("AttractionType")['Rating'].value_counts().reset_index(name='No_Users')

In [ ]:
rating = px.bar(pop_att,'AttractionType','No_Users',hover_data='Rating',color='Rating')
rating.show()

In [ ]:
#Correlation

In [ ]:
correlation = main_data_ml[['UserId','ContinentId','Continent', 'VisitYear', 'VisitMonth','RegionId','Region', 'AttractionCityId', 'AttractionTypeId',
                         'Attraction','AttractionType','CountryId','Country','CityId', 'CityName','VisitModeId','VisitMode','Rating']]


In [ ]:
# Plot heatmap for correlation
plt.figure(figsize=(20,20))
sns.heatmap(correlation.corr(), annot=True,vmax=1,vmin=-1)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
#Distribution of ratings across different attractions and regions

In [ ]:
att_reg_rat = main_data[['Attraction','Region','Rating']]


In [ ]:
rating_filter = att_reg_rat.groupby(['Attraction','Region'])['Rating'].value_counts().reset_index(name='ratted_users').sort_values(by="ratted_users", ascending=False)

In [ ]:
fig = px.bar(rating_filter,'Region','Rating',hover_data=['Attraction','ratted_users'],color='Rating')
fig.show()

# **Machine Learning**

# *Regression*

In [ ]:
reg_mod_df = main_data_ml[[ 'VisitMonth', 'CityName','Attraction','VisitMode','Rating']]

In [ ]:
# Features and target

x =reg_mod_df.drop(columns='Rating',axis=1)
y =reg_mod_df['Rating']

In [ ]:
# Splitting data into train and test

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
# Splitting data into train and test

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
#LGBMRegressor
lgm = LGBMRegressor()
lgm.fit(x_train,y_train)

In [ ]:
#LinearRegression

lr=LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
#Ridge,Lasso

ridge_model = Ridge(alpha=1.0)
lasso_model = Lasso(alpha=1.0)
ridge_model.fit(x_train, y_train)
lasso_model.fit(x_train, y_train)

In [ ]:
#KNeighborsRegressor

KNN=KNeighborsRegressor(n_neighbors=20)
KNN.fit(x_train,y_train)

In [ ]:
#DecisionTreeRegressor model

dtr=DecisionTreeRegressor()
dtr.fit(x_train,y_train)

In [ ]:
#RandomForestRegressor
rfc=RandomForestRegressor(n_estimators=1000, max_depth=100, min_samples_split=100, min_samples_leaf=20,  max_features='sqrt', bootstrap=True)
rfc.fit(x_train,y_train)

In [ ]:
#Hyperparameter tuning

linear_param={'fit_intercept':[True,False]}

decision_param={"splitter":['best', 'random'],"max_depth":[None,5,10,15,20],"min_samples_split":[2,3,4,5,6,7,8,10]}

knn_param={'n_neighbors':[5,7,9,12,15,18,20],"weights":['uniform','distance']}

#Creating multiple models

grid_search_lr=GridSearchCV(lr,linear_param,cv=5)
grid_search_dtr=GridSearchCV(dtr,decision_param,cv=5)
grid_search_KNN=GridSearchCV(KNN,knn_param,cv=5)

#Training multiple models

grid_search_lr.fit(x_train,y_train)
grid_search_dtr.fit(x_train,y_train)
grid_search_KNN.fit(x_train,y_train)

In [ ]:
# Evaluation metrics

y_pred_lr=lr.predict(x_test)
lr_score=lr.score(x_test,y_test)
linear_mae=mean_absolute_error(y_test,y_pred_lr)
r2_lr = r2_score(y_test,y_pred_lr)

y_pred_rm=ridge_model.predict(x_test)
rm_score=ridge_model.score(x_test,y_test)
rm_mae=mean_absolute_error(y_test,y_pred_rm)
r2_rm = r2_score(y_test,y_pred_rm)

y_pred_lm=lasso_model.predict(x_test)
lm_score=lasso_model.score(x_test,y_test)
lm_mae=mean_absolute_error(y_test,y_pred_lm)
r2_lm = r2_score(y_test,y_pred_lm)

y_pred_knn=KNN.predict(x_test)
KNN_score=KNN.score(x_test,y_test)
knn_mae=mean_absolute_error(y_test,y_pred_knn)
r2_knn = r2_score(y_test,y_pred_knn)

y_pred_dtr=dtr.predict(x_test)
dtr_score=dtr.score(x_test,y_test)
dtr_mae=mean_absolute_error(y_test,y_pred_dtr)
r2_dtr = r2_score(y_test,y_pred_dtr)

y_pred_rfc=rfc.predict(x_test)
rfc_score=rfc.score(x_test,y_test)
rfc_mae=mean_absolute_error(y_test,y_pred_rfc)
r2_rfc = r2_score(y_test,y_pred_rfc)

y_pref_lgm = lgm.predict(x_test)
lgm_score=lgm.score(x_test,y_test)
lgm_mae=mean_absolute_error(y_test,y_pref_lgm)
r2_lgm = r2_score(y_test,y_pref_lgm)

print(f"Linear regression \n r2 score : {r2_lr}\n mae : {linear_mae} \nRidge model \n r2 score : {r2_rm}\n mae : {rm_mae}  \nlasso_model \n r2 score : {r2_lm} \n mae : {lm_mae} \nKNN model \n r2 score : {r2_knn}\n mae : {knn_mae} \nDecision tree  \n r2 score : {r2_dtr}\n mae : {dtr_mae} \nRandom forest \n r2 score : {r2_rfc} \n mae : {rfc_mae} \nLGM \n r2 score : {r2_lgm}\n mae : {lgm_mae}")

In [ ]:
#classification

log_param_reg={
            'C': [0.1,1, 10],
            'penalty': ['l1', 'l2']
        }

dtc_param_reg={
            'max_depth': [None,10, 20],
            'min_samples_split': [2, 5]
        }

knnc_param_reg={
            'n_neighbors': [5,10,20],
            'weights': ['uniform', 'distance']}
rfc_param_reg ={ 'n_estimators': [50,100,200],
            'max_depth': [None,10,20],
            "min_samples_split":[5,6,7,8,10]}
#Creating multiple models

re_log=LogisticRegression()
reg_dtc=DecisionTreeClassifier()
reg_knnc=KNeighborsClassifier()
reg_rfc=RandomForestClassifier()

grid_search_log_reg=GridSearchCV(re_log,log_param_reg,cv=5)
grid_search_dtc_reg=GridSearchCV(reg_dtc,dtc_param_reg,cv=5)
grid_search_KNNC_reg=GridSearchCV(reg_knnc,knnc_param_reg,cv=5)
grid_search_rfc_reg=GridSearchCV(reg_rfc,rfc_param_reg,cv=5)


#Training multiple models

grid_search_log_reg.fit(x_train,y_train)
grid_search_dtc_reg.fit(x_train,y_train)
grid_search_KNNC_reg.fit(x_train,y_train)
grid_search_rfc_reg.fit(x_train,y_train)

In [ ]:
# Evaluation metrics

y_pred_log_reg=grid_search_log_reg.predict(x_test)
log_reg_rep=classification_report(y_test,y_pred_log_reg)


y_pred_knnc_reg=grid_search_KNNC_reg.predict(x_test)
knnc_reg_rep=classification_report(y_test,y_pred_knnc_reg)


y_pred_dtc_reg=grid_search_dtc_reg.predict(x_test)
dtc_reg_rep=classification_report(y_test,y_pred_dtc_reg)


y_pred_rfc_reg=grid_search_rfc_reg.predict(x_test)
rfclass_reg_rep=classification_report(y_test,y_pred_rfc_reg)

print(f"Logistic regression \n{log_reg_rep} \nKNN Classifier \n {knnc_reg_rep} \nDecision tree Classifier \n {dtc_reg_rep} \nRandom forest Classifier \n {rfclass_reg_rep} ")

In [ ]:
#Finalized model for classification

Final_RFC_ratings=RandomForestClassifier(max_depth=10, min_samples_split=6, n_estimators=50)
Final_RFC_ratings.fit(x_train,y_train)

In [ ]:
#Final report

y_pred_FRR=Final_RFC_ratings.predict(x_test)
print(classification_report(y_test,y_pred_FRR))

In [ ]:
# Save the model

with open("RFC_Ratings.pkl",'wb') as RFC_file:
    pickle.dump(Final_RFC_ratings,RFC_file)

# *Classification*

In [ ]:
class_data = main_data_ml[['VisitYear','VisitMonth','Rating','Attraction','AttractionType','CityName','Country','VisitMode']]


In [ ]:
# Features and target

x_class = class_data.drop('VisitMode',axis=1)
y_class = class_data['VisitMode']

In [ ]:
# Splitting data into train and test

x_train_class,x_test_class,y_train_class,y_test_class = train_test_split(x_class,y_class,test_size=0.2,random_state=42)

In [ ]:
# Hyperparameter Tuning

log_param={
            'C': [0.1, 1, 10],
            'penalty': ['l1', 'l2']
        }

dtc_param={
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]
        }

knnc_param={
            'n_neighbors': [3, 5, 7,10,12,15,20],
            'weights': ['uniform', 'distance']}
rfc_param ={ 'n_estimators': [50, 100],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]}
#Creating multiple models

g_log=LogisticRegression()
g_dtc=DecisionTreeClassifier()
g_knnc=KNeighborsClassifier()
g_rfc=RandomForestClassifier()

grid_search_log=GridSearchCV(g_log,log_param,cv=5)
grid_search_dtc=GridSearchCV(g_dtc,dtc_param,cv=5)
grid_search_KNNC=GridSearchCV(g_knnc,knnc_param,cv=5)
grid_search_rfc=GridSearchCV(g_rfc,rfc_param,cv=5)

#Training multiple models

grid_search_log.fit(x_train_class,y_train_class)
grid_search_dtc.fit(x_train_class,y_train_class)
grid_search_KNNC.fit(x_train_class,y_train_class)
grid_search_rfc.fit(x_train_class,y_train_class)

In [ ]:
#KNeighborsClassifier
knn_class=KNeighborsClassifier(n_neighbors=20, weights='distance')
knn_class.fit(x_train_class,y_train_class)

In [ ]:
#DecisionTreeClassifier

dtc=DecisionTreeClassifier(max_depth=10)
dtc.fit(x_train_class,y_train_class)

In [ ]:
# RandomForestClassifier

gs_rfc=RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=50)
gs_rfc.fit(x_train_class,y_train_class)

In [ ]:
# Evaluation metrics

#y_pred_log=g_log.predict(x_test_class)
#log_rep=classification_report(y_test_class,y_pred_log)


y_pred_knnc=knn_class.predict(x_test_class)
knnc_rep=classification_report(y_test_class,y_pred_knnc)


y_pred_dtc=dtc.predict(x_test_class)
dtc_rep=classification_report(y_test_class,y_pred_dtc)


y_pred_rfclass=gs_rfc.predict(x_test_class)
rfclass_rep=classification_report(y_test_class,y_pred_rfclass)

print(f"KNN Classifier \n {knnc_rep} \nDecision tree Classifier \n {dtc_rep} \nRandom forest Classifier \n {rfclass_rep} ")

In [ ]:
#Finalized model

from sklearn.neighbors import KNeighborsClassifier
knn_class=KNeighborsClassifier(n_neighbors=20, weights='distance')
knn_class.fit(x_train_class,y_train_class)

In [ ]:
# Save the model

with open("KNNClassifier_VM.pkl",'wb') as KNN_file:
    pickle.dump(knn_class,KNN_file)

# ***Recommendation***

In [ ]:
#Collaborative Filtering

In [ ]:
#user related recommentation

coll_rec = main_data[['UserId','CityId','Rating','CityName','Country']]

In [ ]:
# Load into Surprise format
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(coll_rec[['UserId','CityId','Rating']], reader)
train_data=data_surprise.build_full_trainset()

model = SVD()
model.fit(train_data)

model_results = cross_validate(model,data_surprise,measures=['RMSE','MAE'],cv=5,verbose=True)


In [ ]:
with open("SVD.pkl",'wb') as SVD_file:
    pickle.dump(model,SVD_file)

In [ ]:
def recommend_svd(user_id, n=10):
    all_cities = coll_rec['CityId'].unique()
    rated = coll_rec[coll_rec['UserId'] == user_id]['CityId'].tolist()
    unrated = [city for city in all_cities if city not in rated]

    preds = [model.predict(user_id, cid) for cid in unrated]
    preds.sort(key=lambda x: x.est, reverse=True)

    top_cities = [pred.iid for pred in preds[:n]]
    return coll_rec[coll_rec['CityId'].isin(top_cities)][['CityName', 'Country']].drop_duplicates()


In [ ]:
#Content-Based Filtering

In [ ]:
content_rec=main_data[['CityName','VisitMode','AttractionType','Attraction','Country']].value_counts().reset_index()

In [ ]:
content_rec['rec_data'] =  content_rec['AttractionType'] + ' ' + content_rec['Attraction'] + ' ' + content_rec['Country']

In [ ]:
#Vectorizer

tfid = TfidfVectorizer()

x_rec = tfid.fit_transform(content_rec['rec_data'] )

In [ ]:
#cosine_similarity

co_sim = cosine_similarity(x_rec,x_rec)

In [ ]:
def recommend_place(AttractionType):
  idx = content_rec[content_rec['AttractionType']==AttractionType].index
  idx=idx[0]
  sim = list(enumerate(co_sim[idx]))
  sim = sorted(sim,key=lambda x:x[1],reverse=True)[:50]
  att=[i[0] for i in sim]
  return content_rec.iloc[att][['CityName','VisitMode','Country']]